In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Download the data file
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data -O wdbc.data

--2025-08-25 13:50:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
ERROR: cannot verify archive.ics.uci.edu's certificate, issued by ‘CN=E6,O=Let's Encrypt,C=US’:
  Issued certificate has expired.
To connect to archive.ics.uci.edu insecurely, use `--no-check-certificate'.


In [ ]:
# Define the column names for the dataset
column_names = [
    'ID', 'Diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
    'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
    'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
    'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
    'fractal_dimension_se', 'radius_worst', 'texture_worst',
    'perimeter_worst', 'area_worst', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'concave points_worst',
    'symmetry_worst', 'fractal_dimension_worst'
]

# Load the data into a pandas DataFrame
df = pd.read_csv('wdbc.data', names=column_names)

# Display the first few rows of the DataFrame
display(df.head())

,ID,Diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst


In [ ]:
df.shape

(0, 32)

In [ ]:
df.head()

,ID,Diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst


**TRAIN TEST SPLIT**

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

**SCALING**

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
x_train

In [ ]:
y_train

*LABEL ENCODING*

In [ ]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [ ]:
y_train

In [ ]:
y_test

**NUMPY Arrays to Pytorch Tensors**

In [ ]:
x_train_tensor=torch.from_numpy(x_train)
x_test_tensor=torch.from_numpy(x_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [ ]:
x_train_tensor.shape

In [ ]:
y_train_tensor.shape

**Defining The Model**

In [ ]:
class MySimpleNN():
  def __init__(self,x):
           self.weights=torch.rand(x.shape[1],1,dtype=torch.float64,requires_grad=True)
           self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)
  def forward(self,x):
    z=torch.matmul(x,self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    #clamp prediction to avoid log
    epsilon=1e-7
    y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
    #Calculate loss
    loss=-(y_train_tensor*torch.log(y_pred)+(1-y_train_tensor)*torch.log(1-y_pred)).mean()
    return loss # Add this line to return the loss

    # backward pass
    loss.backward()

    #parameter update
    model.weight.grad.zero_()
    model.bias.grad.zero_()

**IMPORTANT PARAMETER**

In [ ]:
learning_rate=0.1
epochs=1

**Training the Pipeline**

In [ ]:
#Create the model
model=MySimpleNN(x_train_tensor)
#define loop
for epoch in range(epochs):

  #forward pass
  y_pred=model.forward(x_train_tensor)
  print(y_pred)
  #loss calculation
  loss=model.loss_function(y_pred,y_train_tensor)
  print(f'Eoch:{epoch},Loss:{loss.item()}')
  #backward pass
  loss.backward()

  #parameter update
  with torch.no_grad():
      model.weights -= learning_rate * model.weights.grad
      model.bias -= learning_rate * model.bias.grad
  #clear gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

In [ ]:
model.bias

**EVALUATION **

In [ ]:
#MODEL EVALUATION
with torch.no_grad():
  y_pred=model.forward(x_test_tensor)
  y_pred=(y_pred>0.5).float()
  accuracy=(y_pred==y_test_tensor).float().mean
print(f'Accuracy:{accuracy.item()}')

In [ ]:
# Make predictions on the test set
with torch.no_grad():
    y_pred_test = model.forward(x_test_tensor)
    y_pred_test_classes = (y_pred_test > 0.5).int()

# Calculate accuracy
correct_predictions = (y_pred_test_classes == y_test_tensor.unsqueeze(1)).sum().item()
accuracy = correct_predictions / len(y_test_tensor)

print(f"Accuracy on the test set: {accuracy:.4f}")